In [ ]:
import pandas as pd
import optuna
import numpy as np
import lightgbm as lgb
import pickle

df = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
final = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')
original = pd.read_csv('/kaggle/input/bank-marketing-dataset-full/bank-full.csv', sep=';')

original['y'] = original['y'].map({'no': 0, 'yes': 1})

def simplify_contact(x):
    if x == 'unknown':
        return 'unknown'
    else:
        return 'known_contact'

def preprocessing(df):

    df['job_edu'] = df['job'].astype(str) + "_" + df['education'].astype(str)
    df = pd.get_dummies(df, columns=['job_edu'], prefix='job_edu')
    df.drop('job', axis=1, inplace=True)
    df.drop('education', axis=1, inplace=True)

    df['balance_log'] = np.log1p(df['balance'].clip(lower=0))
    
    df = pd.get_dummies(df, columns=['marital'], prefix='marital')
    df['default'] = df['default'].map({'yes': 1, 'no': 0})
    df['housing'] = df['housing'].map({'yes': 1, 'no': 0})
    df['loan'] = df['loan'].map({'yes': 1, 'no': 0})

    df['contact_simple'] = df['contact'].apply(simplify_contact)
    df = pd.get_dummies(df, columns=['contact_simple'], prefix='contact')
    df.drop('contact', axis=1, inplace=True)

    df['age^2'] = df["age"]**2
    
    df['prev_camp'] = (df['pdays'] != -1).astype(int)
    df['pdays'] = df['pdays'].replace(-1, 999)
    df = pd.get_dummies(df, columns=['poutcome'], prefix='poutcome') # has unknown. but means still ongoing 

    df['duration_sin'] = np.sin(2*np.pi * df['duration'] / 400)
    df['duration_cos'] = np.cos(2*np.pi * df['duration'] / 400)

    month_map = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    }
    df['month_num'] = df['month'].map(month_map).astype('int')

    df['month_sin'] = np.sin(2 * np.pi * df['month_num'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month_num'] / 12)

    df.drop('month_num',axis=1,inplace=True)

    df = pd.get_dummies(df, columns=['month'], prefix='month')

    return df

def FE(X, test, c, original, target='y'):
    new_col = f"{c}_mean_target_orig"
    
    target_map = original.groupby(c)[target].mean()
    mapping_count = original[c].value_counts()
    
    global_mean = original[target].mean()
    
    X[f"{c}_count"] = X[c].map(mapping_count).fillna(0)
    test[f"{c}_count"] = test[c].map(mapping_count).fillna(0)
    
    X[new_col] = X[c].map(target_map).fillna(global_mean)
    test[new_col] = test[c].map(target_map).fillna(global_mean)
    
    return X, test

def extra(train, test):
    combined = pd.concat([train, test], axis=0, ignore_index=True)
    
    def f1(x):
        if x['education']=='unknown' and x['contact']=='unknown' and x['poutcome']=='unknown':
            return 21
        if (x['education']=='unknown' and x['contact']=='unknown') \
           or (x['education']=='unknown' and x['poutcome']=='unknown') \
           or (x['contact']=='unknown' and x['poutcome']=='unknown'):
            return 7
        if x['education']=='unknown' or x['contact']=='unknown' or x['poutcome']=='unknown':
            return 3
        return 0
    
    def f2(x):
        if x['default']=='no' and x['housing']=='no' and x['loan']=='no':
            return 21
        if (x['default']=='no' and x['housing']=='no') \
           or (x['default']=='no' and x['loan']=='no') \
           or (x['housing']=='no' and x['loan']=='no'):
            return 7
        if x['default']=='no' or x['housing']=='no' or x['loan']=='no':
            return 3
        return 0
    
    combined['unknown_score'] = combined.apply(f1, axis=1)
    combined['many_no_score'] = combined.apply(f2, axis=1)
    
    unknown_freq = combined['unknown_score'].value_counts().to_dict()
    many_no_freq = combined['many_no_score'].value_counts().to_dict()
    
    combined['unknown_score_freq'] = combined['unknown_score'].map(unknown_freq)
    combined['many_no_score_freq'] = combined['many_no_score'].map(many_no_freq)
    
    train_len = len(train)
    train['unknown_score_freq'] = combined.loc[:train_len-1, 'unknown_score_freq'].values
    test['unknown_score_freq'] = combined.loc[train_len:, 'unknown_score_freq'].values
    
    train['many_no_score_freq'] = combined.loc[:train_len-1, 'many_no_score_freq'].values
    test['many_no_score_freq'] = combined.loc[train_len:, 'many_no_score_freq'].values
    
    return train, test

COLS = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome',]

df.drop('id', axis=1, inplace=True)
final_id = final['id']
final.drop('id', axis=1, inplace=True)

for c in COLS:
    df, test = FE(df, final, c, original)

df, final = extra(df, final)

df = preprocessing(df)
final = preprocessing(final)

y = df['y']
X = df.drop('y', axis=1)

train_data = lgb.Dataset(X, label=y)

def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'is_unbalance': True,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 150),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 10.0),
        'verbose': -1,
        'feature_pre_filter': False,
    }
    
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=4000,
        nfold=5,
        stratified=True,
        shuffle=True,
        seed=42,
        callbacks=[lgb.early_stopping(stopping_rounds=100)],
    )
    
    best_auc = max(cv_results['valid auc-mean'])
    best_rounds = len(cv_results['valid auc-mean'])
    
    trial.set_user_attr("best_rounds", best_rounds)
    return best_auc

pruner = optuna.pruners.MedianPruner(n_warmup_steps=1)
study = optuna.create_study(direction='maximize', pruner=pruner)
study.optimize(objective, timeout=60*60*8)

best_params = study.best_params
best_rounds = study.best_trial.user_attrs["best_rounds"]

model_lgb = lgb.train(
    {**best_params, 'objective': 'binary', 'metric': 'auc', 'is_unbalance': True},
    train_data,
    num_boost_round=best_rounds
)

preds = model_lgb.predict(final)

submission = pd.DataFrame({
    "id": final_id,
    "y": preds
})

submission.to_csv("submission.csv", index=False)

with open("lgb_model.pkl", "wb") as f:
    pickle.dump((model_lgb), f)

[I 2025-08-11 23:30:25,759] A new study created in memory with name: no-name-7182ee16-fd44-4ead-8ebb-f123f8d91623


Training until validation scores don't improve for 100 rounds


[I 2025-08-11 23:36:11,334] Trial 0 finished with value: 0.9737589397366427 and parameters: {'learning_rate': 0.07039908814401957, 'num_leaves': 114, 'max_depth': 10, 'min_child_samples': 105, 'subsample': 0.783583022041211, 'colsample_bytree': 0.6196182243461509, 'reg_alpha': 3.3001243336160755, 'reg_lambda': 1.7605123659053645}. Best is trial 0 with value: 0.9737589397366427.


Early stopping, best iteration is:
[752]	cv_agg's valid auc: 0.973759 + 0.000503667
Training until validation scores don't improve for 100 rounds


[I 2025-08-11 23:53:48,470] Trial 1 finished with value: 0.9703400926633441 and parameters: {'learning_rate': 0.006850033973741573, 'num_leaves': 280, 'max_depth': 4, 'min_child_samples': 113, 'subsample': 0.9201880384114736, 'colsample_bytree': 0.7035050810358845, 'reg_alpha': 0.16539793153851123, 'reg_lambda': 2.8751219748852384}. Best is trial 0 with value: 0.9737589397366427.


...

Early stopping, best iteration is:
[1564]	cv_agg's valid auc: 0.974032 + 0.00052086
Training until validation scores don't improve for 100 rounds


[I 2025-08-12 07:29:06,419] Trial 24 finished with value: 0.9740731563553437 and parameters: {'learning_rate': 0.02908759704645533, 'num_leaves': 230, 'max_depth': 7, 'min_child_samples': 5, 'subsample': 0.5331021520389398, 'colsample_bytree': 0.5262748744969131, 'reg_alpha': 9.956174235593256, 'reg_lambda': 6.747943462592171}. Best is trial 13 with value: 0.9741263356758287.


Early stopping, best iteration is:
[3243]	cv_agg's valid auc: 0.974073 + 0.000457806
Training until validation scores don't improve for 100 rounds


[I 2025-08-12 07:46:41,882] Trial 25 finished with value: 0.97398659362023 and parameters: {'learning_rate': 0.04930625150578331, 'num_leaves': 143, 'max_depth': 6, 'min_child_samples': 31, 'subsample': 0.5981874776806715, 'colsample_bytree': 0.44996159808699215, 'reg_alpha': 7.800093017031082, 'reg_lambda': 4.415053800517421}. Best is trial 13 with value: 0.9741263356758287.


Early stopping, best iteration is:
[2544]	cv_agg's valid auc: 0.973987 + 0.000473944
